# Testing Aigents Sentiment Analysis Web API

In [1]:
import os, sys, math
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

import datetime as dt

if 'pygents.aigents_api' in sys.modules:
    del sys.modules['pygents.aigents_api']
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']

from pygents.aigents_api import AigentsSentiment, PygentsSentiment, TextMetrics
from pygents.util import vector_proximity


def language_metrics(lang,metrics_list):
    metrics = {}
    for m in metrics_list:
        metrics[m] = './data/dict/' + lang + '/' + m + '.txt'
    return metrics

        
p = PygentsSentiment('./data/dict/en/positive.txt',
                     './data/dict/en/negative.txt',debug=True)
a = TextMetrics(language_metrics('en',['catastrophizing','dichotomous-reasoning']),debug=True)
s = TextMetrics(language_metrics('en',['positive','negative']),debug=True)

print(a.gram_arity)
print(s.gram_arity)


4
6


In [2]:
a.metrics


{'catastrophizing': {('will', 'be', 'a', 'catastrophe'),
  ('will', 'be', 'a', 'disaster'),
  ('will', 'be', 'horrible'),
  ('will', 'be', 'impossible'),
  ('will', 'be', 'terrible'),
  ('will', 'end'),
  ('will', 'fail'),
  ('will', 'go', 'wrong'),
  ('will', 'never', 'end'),
  ('will', 'not', 'end'),
  ('will', 'not', 'happen')},
 'dichotomous-reasoning': {('all',),
  ('always',),
  ('black', 'or', 'white'),
  ('either', 'or'),
  ('ever',),
  ('every',),
  ('everybody',),
  ('everyone',),
  ('everything',),
  ('everywhere',),
  ('neither',),
  ('never',),
  ('no', 'one'),
  ('nobody',),
  ('nor',),
  ('not', 'a', 'single'),
  ('nothing',),
  ('nowhere',),
  ('only',),
  ('perfect',),
  ('the', 'best'),
  ('the', 'worst'),
  ('worthless',)}}

In [3]:
for m in s.metrics:
    ngrams = s.metrics[m]
    for n in ngrams:
        if len(n) > 4:
            print(n)


('feel', 'like', 'a', 'million', 'dollars')
('revelation', 'of', 'saint', 'john', 'the', 'divine')
('too', 'big', 'for', "one's", 'breeches')
('pull', 'a', 'fast', 'one', 'on')
('respiratory', 'distress', 'syndrome', 'of', 'the', 'newborn')


In [6]:
texts = ['good news','bad news','good bad news','good and pleasant news about bad things','empty spaces']
for t in texts:
    p1 = p.get_sentiment(t)
    s1 = s.get_sentiment_words(t)
    #assert s1 == s2
    print(p1,s1,t)


(0.85, 0.85, -0.0, 0.0, 2, 1) {'positive': 0.8537850880489681} good news
(-0.85, 0.0, -0.85, 0.0, 2, 1) {'negative': 0.8537850880489681} bad news
(0.0, 0.77, -0.77, 0.77, 3, 1) {'positive': 0.7678579849927549, 'negative': 0.7678579849927549} good bad news
(0.14, 0.74, -0.59, 0.66, 7, 1) {'positive': 0.7354361527213304, 'negative': 0.5921428688354764} good and pleasant news about bad things
(0.0, 0.0, -0.0, 0.0, 2, 1) {} empty spaces


In [ ]:
#TODO rounding
#TODO distortions
#TODO unit

